In [1]:
# импортируем pandas и numpy
import pandas as pd
import numpy as np

# импортируем функцию duplicate_data(), которая
# дублирует датафрейм для всех сегментов
from etna.datasets.utils import duplicate_data

# отключаем предупреждение SettingWithCopyWarning
pd.options.mode.chained_assignment = None

In [2]:
# загружаем набор с эндогенными рядами
endog_df = pd.read_csv('endog_df.csv')
endog_df

,timestamp,segment,target
0,2019-01-01,segment_a,170
1,2019-01-02,segment_a,243
2,2019-01-03,segment_a,267
3,2019-01-04,segment_a,287
4,2019-01-05,segment_a,279
...,...,...,...
1331,2019-11-26,segment_d,941
1332,2019-11-27,segment_d,949
1333,2019-11-28,segment_d,896
1334,2019-11-29,segment_d,905


In [3]:
# задаем горизонт
horizon = 14

# задаем количество периодов, длину набора
# делим на количество сегментов
periods = int(len(endog_df) / endog_df['segment'].nunique())
periods

334

In [4]:
# задаем индекс для будущего набора с эндогенными переменными, длина
# равна длине набора с эндогенными рядами + длина горизонта
exog_index = pd.date_range('2019-01-01', 
                           periods=periods + horizon,
                           freq='D')
exog_index

DatetimeIndex(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04',
               '2019-01-05', '2019-01-06', '2019-01-07', '2019-01-08',
               '2019-01-09', '2019-01-10',
               ...
               '2019-12-05', '2019-12-06', '2019-12-07', '2019-12-08',
               '2019-12-09', '2019-12-10', '2019-12-11', '2019-12-12',
               '2019-12-13', '2019-12-14'],
              dtype='datetime64[ns]', length=348, freq='D')

In [5]:
# добавим экзогенные переменные - начало месяца и конец месяца
month_start = exog_index.is_month_start.astype(int)
month_end = exog_index.is_month_end.astype(int)

In [6]:
# формируем набор экзогенных переменных без привязки к сегментам
exog_df = pd.DataFrame({'timestamp': exog_index.values, 
                        'month_start': month_start,
                        'month_end': month_end})
exog_df

,timestamp,month_start,month_end
0,2019-01-01,1,0
1,2019-01-02,0,0
2,2019-01-03,0,0
3,2019-01-04,0,0
4,2019-01-05,0,0
...,...,...,...
343,2019-12-10,0,0
344,2019-12-11,0,0
345,2019-12-12,0,0
346,2019-12-13,0,0


In [7]:
# получаем сегменты
segments = endog_df['segment'].unique().tolist()
segments

['segment_a', 'segment_b', 'segment_c', 'segment_d']

In [8]:
# получаем итоговый набор экзогенных переменных, просто
# продублируем экзогенные данные по сегментам
final_exog_df = duplicate_data(df=exog_df, 
                               segments=segments, 
                               format='long')
final_exog_df

,timestamp,month_start,month_end,segment
0,2019-01-01,1,0,segment_a
1,2019-01-02,0,0,segment_a
2,2019-01-03,0,0,segment_a
3,2019-01-04,0,0,segment_a
4,2019-01-05,0,0,segment_a
...,...,...,...,...
343,2019-12-10,0,0,segment_d
344,2019-12-11,0,0,segment_d
345,2019-12-12,0,0,segment_d
346,2019-12-13,0,0,segment_d


In [9]:
# формируем набор экзогенных данных для перекрестной 
# проверки, запускаемой на всем наборе, все
# переменные в будущем известны
ts_exog_ex_feats_known = final_exog_df
ts_exog_ex_feats_known

,timestamp,month_start,month_end,segment
0,2019-01-01,1,0,segment_a
1,2019-01-02,0,0,segment_a
2,2019-01-03,0,0,segment_a
3,2019-01-04,0,0,segment_a
4,2019-01-05,0,0,segment_a
...,...,...,...,...
343,2019-12-10,0,0,segment_d
344,2019-12-11,0,0,segment_d
345,2019-12-12,0,0,segment_d
346,2019-12-13,0,0,segment_d


In [10]:
# формируем набор экзогенных данных для базовой модели 
# и перекрестной проверки, запускаемой на обучающем
# наборе, все переменные в будущем известны
train_exog_ex_feats_known = final_exog_df[
    final_exog_df['timestamp'] <= '2019-11-30'
]
train_exog_ex_feats_known

,timestamp,month_start,month_end,segment
0,2019-01-01,1,0,segment_a
1,2019-01-02,0,0,segment_a
2,2019-01-03,0,0,segment_a
3,2019-01-04,0,0,segment_a
4,2019-01-05,0,0,segment_a
...,...,...,...,...
329,2019-11-26,0,0,segment_d
330,2019-11-27,0,0,segment_d
331,2019-11-28,0,0,segment_d
332,2019-11-29,0,0,segment_d


In [11]:
# записываем наборы в CSV-файлы
ts_exog_ex_feats_known.to_csv(
    'ts_exog_ex_feats_known.csv', index=False
)
train_exog_ex_feats_known.to_csv(
    'train_exog_ex_feats_known.csv', index=False
)